In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
model = YOLO("PATH_TO_BEST_PT")

In [ ]:
img_path = "PATH_TO_IMAGE"
img = cv2.imread(img_path)
img_resized = cv2.resize(img, (416, 416))
img_tensor = torch.from_numpy(img_resized).permute(2,0,1).unsqueeze(0).float() / 255.0
img_tensor.requires_grad_()

In [ ]:
preds = model.model(img_tensor)[0]
num_classes = preds.shape[1] - 5

In [ ]:
plt.figure(figsize=(5, 4))
plt.title("Original")
plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
for class_idx in range(num_classes):
    model.model.zero_grad()
    img_tensor.grad = None
    score = preds[0, 5 + class_idx]
    score_scalar = score.max()
    score_scalar.backward(retain_graph=True)
    saliency = img_tensor.grad.data.abs().squeeze().cpu().numpy()
    saliency = np.max(saliency, axis=0)
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
    plt.figure(figsize=(5, 4))
    plt.title(f"Class {class_idx} Saliency")
    plt.imshow(saliency, cmap="hot")
    plt.axis('off')
    plt.show()